In [1]:
# group options based on right, symbol and strike.
# this makes it easy to delete unwanted ones on inspection.
def grp_opts(df):
    '''Groups options and sorts strikes by puts and calls
    Arg: 
       df as dataframe. Requires 'symbol', 'strike' and 'dte' fields in the df
    Returns: sorted dataframe'''
    
    gb = df.groupby('right')

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, False, True])
    else:
        df_calls =  pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, False, False])
    else:
        df_puts =  pd.DataFrame([])

    df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
    
    return df

In [4]:
from os import listdir
import pandas as pd

fspath = '../data/snp/' # path for pickles

minOptPrice = 0.1
minRom = 0.1

fs = listdir(fspath)

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

optsList = [f for f in fs if f[-3:] == 'pkl']

df1 = pd.concat([pd.read_pickle(fspath+f) for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

df2 = df1[((df1.strike > df1.hi52) | (df1.strike < df1.lo52)) & (df1.optPrice > minOptPrice) & (df1.rom > minRom)]

df2 = grp_opts(df2)

In [5]:
df2

,optId,symbol,right,expiration,dte,strike,undPrice,lo52,hi52,Fall,Rise,loFall,hiRise,std3,loStd3,hiStd3,lotsize,optPrice,optMargin,rom
0,360782514.0,MMM,P,20190531,45.0,170.0,216.63,176.87,259.77,53.23,41.59,163.40,258.22,27.281231,189.348769,243.911231,1.0,0.25,8.08,0.173267
1,360782499.0,MMM,P,20190531,45.0,165.0,216.63,176.87,259.77,53.23,41.59,163.40,258.22,27.281231,189.348769,243.911231,1.0,0.13,5.08,0.143307
2,360133188.0,MMM,P,20190524,38.0,175.0,216.63,176.87,259.77,51.66,41.59,164.97,258.22,27.281231,189.348769,243.911231,1.0,0.27,15.10,0.118578
